# SpikeFinder
Finds fast price spikes in a list of stock tickers.
Created     2023-04-07
Last Update 2023/04/09

Search for the SPAC or shellco that DB will use for IPO merger.
1. Use price spike as indicator.
2. Use list of 760 SPACs and shellcos as suspects.

Spike Definition: Price jumps > MIN_SPIKE_SIZE in fewer than MAX_SPIKE_DAYS.

User controls
1. Stock ticker list. Add, delete tickers. Enclose each ticker with single quotes, followed by a comma, as shown.
2. Set MIN_SPIKE_SIZE as fraction of stock price at start of spike. Price must increase by this fraciton within MAX_SPIKE_DAYS
3. Set MAX_SPIKE_DAYS = Spike must occur within MAX_SPIKE_DAYS. This is an integer.
4. Set DAYS_BACK = Number of trading days to look back for spike.
5. Plot stock price and pivots with npZigZagPlotter.


# TODO
Developer ToDo list
1. Build plotter from npZigZagPlotter - Done 04/07/23
2. Check other stock indicators in stock_indicator.
3. Fix datetime foolishness to convert integer and float numbers to datetime object.
4. Finish SpikeFinder:
5. Pivot = 1 if price change > MIN_SPIKE_SIZE in <= MAX_SPIKE_DAYS
6. Pivot = -1 if price change < -MIN_spike_SIZE in <=MAX_SPIKE_Dys
7. else Pivot = 0
8. Output is:
8.1 List of ticker with spikes, up and down.
8.2 pivots for each ticker. Send the pivot list to npZigZagPlotter

In [1]:
# **********
#
#SpikeFinder - created 04/06/23
#    Goal - To identify spikes in stock prices.
#
#     Steps
#          1. Initialize libraries, constants, etc.
#          2. Get prices for selected tickers.
#          3. Calculate price_change delta for date D from date D - MAX_SPIKE DAYS
#          4. If delta >= MIN_SPIKE_SIZE then tag this day as a spike
#          5. If delta > 0 for D + 1, tag this day as a spike also.
#
# **********

# **********
#
# User sets constants
#
# **********

# Set DAR_key
DAR_key = '90180f15ecc74513a01ca017eca2bb4f'
# Set TIKCERS = ('XXX', 'YYY')
TICKERS = ('NWBO', 'SANA')

# Set MIN_SPIKE_SIZE = 0.15  #fraction between 0 and 1
MIN_SPIKE_SIZE = 0.25

# Set MAX_SPIKE_DAYS = integer for how fast the spike must occur.
MAX_SPIKE_DAYS = 4

# Set DAYS_BACK = Number of trding days to look back for spike.
DAYS_BACK = 100 # approx. 5 months. Note DAYS_BACK uses calendar days, not just trading days.

# **********
#
# Import LIbraries
#
# **********

import csv
import copy
import json
import numpy as np
import os
import pandas as pd
import requests   # for http requests
import scipy
from   scipy import stats
from   scipy.stats import norm
import time
import yfinance as yf

import datetime
from   datetime import datetime
from   datetime import date
#from   datetime import fromtimestamp
from   dateutil.relativedelta import relativedelta

# **********
#
# Get stock prices
#
# **********
#Test conversion of float and integer to datetime object.
#this_date_int = 1680048000
#this_date_float = float(this_date_int)
#print(this_date_int, ' = ', str(this_date_int))
#this_date = datetime(this_date_float)
#print(this_date, ' same as ', this_date.strftime('%Y-%m-%d'))


ticker_prices = pd.DataFrame(columns = ['Date', 'Ticker', 'price'])

#new_months = my_datetime - relativedelta(months = 4)
TODAY = datetime.today()
print('TODAY = ', TODAY)
ts = datetime.timestamp(TODAY)
print(ts)
date_only = int(ts)
print(date_only)

end_date = int(datetime.timestamp(TODAY - relativedelta(days = 1)))
print('end_date = ', end_date)
end_date_str = str(end_date)
start_date = int(datetime.timestamp(TODAY - relativedelta(days = DAYS_BACK)))
print('start_date = ', start_date)
start_date_str = str(start_date)

SPAC_df = pd.read_csv('SPAC_List.csv')
TICKERS = SPAC_df.iloc[0:3]

tickers_pivots = pd.DataFrame()
tickers_pivots['Symbol'] = TICKERS.Symbol
tickers_pivots['pivot'] = 0

# Pivot Detector
# tickers_pivots['ticker'] = ticker_list
# For each ticker Get prices. Create cols for pivot and prices offset by X days
#      Initialize ticker['pivot'] = 0. Calculate pivot value. Merge each ticker_df with tickers_pivots
#      Create pivot_df with only rows where pivot value changed. Merge with ticker_pivot and add rows with new dates.
# Drop price offset cols. 
# 

for ticker in TICKERS:
    
    ticker_df = pd.DataFrame()
    price_rqst = 'https://api.darqube.com/data-api/market_data/historical/' + ticker + '?token=' + DAR_key + \
                    '&start_date=' + start_date_str + '&end_date=' + end_date_str + '&interval=1d'
    
    #price_rqst = 'https://api.darqube.com/data-api/market_data/quote/' + ticker + '?token=' + DAR_key
    response = requests.get(price_rqst)
    price_dict = response.json()
    ticker_prices = pd.DataFrame(data = price_dict)
    print(ticker_prices['time'])
    ticker_prices['Date'] = pd.to_datetime(ticker_prices['time'].astype(str))
    tickers_pivots = tickers_pivots.merge(ticker_prices)
    
    #ticker_prices.drop(columns = 'time', inplace = True)
    #Convert 'time' col to date string of form YYYY-MM-DD
    
    # Pivot detector
    # Initialize cols for comparison
    for i in range(MAX_SPIKE_DAYS):
        col_name = 'price-' +str(i)
        tickers_pivot[col_name] = ticker_pivot['adjusted_close'].offset(periods = i)
        
    #
        


    
    #Set date as index
    #Find spikes
    # 1. Find min price in days from (this_day - MAX_SPIKE_DAYS to this_day
    # 2. if (price[this_day] - price[min_day]) / price[min_day] > MIN_SPIKE_SIZE then df['SPike_Day]=True']
    # 3. Find last day of spike. 

print('thats all folks')




1680048000  =  1680048000


TypeError: 'float' object cannot be interpreted as an integer